<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

## Step 1.1: Configure the environment

In [2]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
# from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, load_s1_dem, scale, calculate_band_indices, apply_mask, mask_water, all_masks, do_prediction

In [3]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [4]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "tongatapu"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}_postcard_4"
print(version)

nm-tongatapu-11122025_postcard_4


In [5]:
gdfs = []
postcards_path = "training-data/"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [6]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-tongatapu-11122025_postcard_4.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [7]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [8]:
training = gpd.read_file(f"training-data/{version}.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [9]:
training

,LULC_code,LULC_class,geometry
0,6,Water,POINT (-175.32648 -21.05595)
1,6,Water,POINT (-175.36156 -21.09345)
2,6,Water,POINT (-175.12939 -21.27945)
3,6,Water,POINT (-175.02447 -21.1477)
4,6,Water,POINT (-175.31365 -21.16867)
...,...,...,...
985,4,Settlements,POINT (-175.1994 -21.13518)
986,4,Settlements,POINT (-175.19913 -21.13671)
987,4,Settlements,POINT (-175.20174 -21.13612)
988,4,Settlements,POINT (-175.20039 -21.13224)


In [10]:
print(training['LULC_class'].value_counts())
print('total gps points',(len(training)))

LULC_class
Grazing_Cropland    310
Forest_land         200
Settlements         140
Wetland             120
Water               110
Bare_land           110
Name: count, dtype: int64
total gps points 990


In [11]:
datetime = "2023"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2023


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 359MB
Dimensions:      (y: 2654, x: 3753, time: 1)
Coordinates:
  * y            (y) float64 21kB -2.383e+06 -2.383e+06 ... -2.41e+06 -2.41e+06
  * x            (x) float64 30kB 3.856e+06 3.856e+06 ... 3.893e+06 3.893e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2023-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 40MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 40MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 20MB

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
# scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled = calculate_band_indices(scaled)
scaled

<xarray.Dataset> Size: 2GB
Dimensions:        (y: 2654, x: 3753)
Coordinates:
  * y              (y) float64 21kB -2.383e+06 -2.383e+06 ... -2.41e+06
  * x              (x) float64 30kB 3.856e+06 3.856e+06 ... 3.893e+06 3.893e+06
    spatial_ref    int32 4B 3832
    time           datetime64[ns] 8B 2023-01-01
Data variables: (12/32)
    nir            (y, x) float64 80MB 0.0423 0.0435 0.0455 ... 0.0384 0.0389
    red            (y, x) float64 80MB 0.0462 0.0461 0.0494 ... 0.0409 0.0434
    blue           (y, x) float64 80MB 0.0625 0.0622 0.0646 ... 0.0607 0.0602
    green          (y, x) float64 80MB 0.05 0.05 0.0524 ... 0.0454 0.046 0.047
    emad           (y, x) float32 40MB 0.1268 0.1316 0.1361 ... 0.1186 0.1188
    smad           (y, x) float32 40MB 1.771e-06 1.88e-06 ... 4.147e-06
    ...             ...
    ndci           (y, x) float64 80MB 0.01911 0.03758 ... 0.03424 0.002299
    nbi            (y, x) float64 80MB 0.002358 0.01695 ... 0.01665 0.007653
    ndmi           (y, x) float64 80MB -0.002358 -0.01695 ... -0.01665 -0.007653
    bsi            (y, x) float64 80MB -0.08883 -0.07528 ... -0.116 -0.1023
    awei           (y, x) float64 80MB 0.07067 0.06565 ... 0.06795 0.06943
    tc_wetness     (y, x) float64 80MB 0.0227 0.02044 ... 0.02145 0.02252

In [17]:
scaled, mask = all_masks(scaled, return_mask = True)
# mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [28]:
scaled

<xarray.Dataset> Size: 2GB
Dimensions:        (y: 2654, x: 3753)
Coordinates:
  * y              (y) float64 21kB -2.383e+06 -2.383e+06 ... -2.41e+06
  * x              (x) float64 30kB 3.856e+06 3.856e+06 ... 3.893e+06 3.893e+06
    spatial_ref    int32 4B 3832
    time           datetime64[ns] 8B 2023-01-01
Data variables: (12/32)
    nir            (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    red            (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    blue           (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    green          (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    emad           (y, x) float32 40MB nan nan nan nan nan ... nan nan nan nan
    smad           (y, x) float32 40MB nan nan nan nan nan ... nan nan nan nan
    ...             ...
    ndci           (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    nbi            (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    ndmi           (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    bsi            (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    awei           (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan
    tc_wetness     (y, x) float64 80MB nan nan nan nan nan ... nan nan nan nan

In [18]:
# scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [29]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(scaled.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    scaled.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,ndci,nbi,ndmi,bsi,awei,tc_wetness,y,x,spatial_ref,time
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2383205.0,3859835.0,3832,2023-01-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2387655.0,3855935.0,3832,2023-01-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2409735.0,3881775.0,3832,2023-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2394095.0,3893455.0,3832,2023-01-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-2396575.0,3861265.0,3832,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,0.2348,0.2107,0.1506,0.1891,0.047303,8.195043e-08,0.000003,0.2388,0.2563,0.3130,...,0.038997,0.142753,-0.142753,0.148946,-0.120950,-0.053969,-2392605.0,3873985.0,3832,2023-01-01
986,0.3164,0.2806,0.2096,0.2473,0.079557,5.739927e-08,0.000003,0.3615,0.2947,0.3866,...,0.106227,0.099858,-0.099858,0.086250,-0.140475,-0.002637,-2392785.0,3874015.0,3832,2023-01-01
987,0.2451,0.2180,0.1563,0.1883,0.046965,1.190364e-07,0.000003,0.2389,0.2742,0.3012,...,0.018681,0.102691,-0.102691,0.119061,-0.119425,-0.037975,-2392715.0,3873725.0,3832,2023-01-01


In [30]:
training_values.columns

Index(['nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad', 'nir08',
       'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'swir22_swir16', 'mci', 'ndci', 'nbi', 'ndmi', 'bsi',
       'awei', 'tc_wetness', 'y', 'x', 'spatial_ref', 'time'],
      dtype='object')

In [31]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["LULC_code"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,LULC_code,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,ndci,nbi,ndmi,bsi,awei,tc_wetness,y,x,spatial_ref,time
34,4,0.3070,0.1063,0.0852,0.1147,0.079907,1.329660e-07,0.000005,0.3332,0.3604,...,0.182622,-0.125986,0.125986,-0.162418,-0.401925,-0.066308,-2393675.0,3876145.0,3832,2023-01-01
35,2,0.3419,0.0842,0.0633,0.0989,0.156016,6.550729e-07,0.000010,0.3326,0.3506,...,0.281876,-0.234519,0.234519,-0.320515,-0.489225,-0.047389,-2404505.0,3875165.0,3832,2023-01-01
36,2,0.2105,0.0710,0.0717,0.0841,0.170746,1.638007e-06,0.000012,0.2695,0.3155,...,0.303238,-0.053027,0.053027,-0.216642,-0.290075,-0.031707,-2404585.0,3875195.0,3832,2023-01-01
37,2,0.3322,0.1085,0.0794,0.1169,0.133435,6.234050e-07,0.000008,0.3098,0.1713,...,0.200737,-0.115327,0.115327,-0.198951,-0.480275,-0.067273,-2404405.0,3875085.0,3832,2023-01-01
38,2,0.2817,0.1015,0.0749,0.1016,0.130518,6.958604e-07,0.000008,0.2729,0.2862,...,0.167692,-0.062017,0.062017,-0.149952,-0.426325,-0.072172,-2404345.0,3875065.0,3832,2023-01-01


In [32]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

37 total columns
columns included Index(['LULC_code', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'swir22_swir16', 'mci', 'ndci', 'nbi', 'ndmi', 'bsi',
       'awei', 'tc_wetness', 'y', 'x', 'spatial_ref', 'time'],
      dtype='object')


In [33]:
print(training_array['LULC_code'].value_counts())
print('total gps points',(len(training_array)))

LULC_code
2    310
1    197
4    138
3    110
5    102
6     25
Name: count, dtype: int64
total gps points 882


In [34]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [35]:
training_array

,LULC_code,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,swir22_swir16,mci,ndci,nbi,ndmi,bsi,awei,tc_wetness,y,x
34,4,0.3070,0.1063,0.0852,0.1147,0.079907,1.329660e-07,0.000005,0.3332,0.3604,...,0.739824,1.996099,0.182622,-0.125986,0.125986,-0.162418,-0.401925,-0.066308,-2393675.0,3876145.0
35,2,0.3419,0.0842,0.0633,0.0989,0.156016,6.550729e-07,0.000010,0.3326,0.3506,...,0.586321,2.274784,0.281876,-0.234519,0.234519,-0.320515,-0.489225,-0.047389,-2404505.0,3875165.0
36,2,0.2105,0.0710,0.0717,0.0841,0.170746,1.638007e-06,0.000012,0.2695,0.3155,...,0.584786,1.585090,0.303238,-0.053027,0.053027,-0.216642,-0.290075,-0.031707,-2404585.0,3875195.0
37,2,0.3322,0.1085,0.0794,0.1169,0.133435,6.234050e-07,0.000008,0.3098,0.1713,...,0.631879,2.038037,0.200737,-0.115327,0.115327,-0.198951,-0.480275,-0.067273,-2404405.0,3875085.0
38,2,0.2817,0.1015,0.0749,0.1016,0.130518,6.958604e-07,0.000008,0.2729,0.2862,...,0.651527,1.978230,0.167692,-0.062017,0.062017,-0.149952,-0.426325,-0.072172,-2404345.0,3875065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,4,0.2348,0.2107,0.1506,0.1891,0.047303,8.195043e-08,0.000003,0.2388,0.2563,...,0.989137,1.030729,0.038997,0.142753,-0.142753,0.148946,-0.120950,-0.053969,-2392605.0,3873985.0
986,4,0.3164,0.2806,0.2096,0.2473,0.079557,5.739927e-08,0.000003,0.3615,0.2947,...,0.891619,0.911028,0.106227,0.099858,-0.099858,0.086250,-0.140475,-0.002637,-2392785.0,3874015.0
987,4,0.2451,0.2180,0.1563,0.1883,0.046965,1.190364e-07,0.000003,0.2389,0.2742,...,0.969124,1.083076,0.018681,0.102691,-0.102691,0.119061,-0.119425,-0.037975,-2392715.0,3873725.0
988,4,0.3333,0.2749,0.1864,0.2248,0.148366,7.668376e-07,0.000006,0.3386,0.3295,...,0.917577,1.036058,0.078445,0.071458,-0.071458,0.094205,-0.240225,-0.030627,-2392255.0,3873875.0


In [36]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/{version}-training.csv", index=False)

In [37]:
training_array["LULC_code"].dtype

dtype('int32')